In [ ]:
%matplotlib inline

In [ ]:
FEATURES = "data/features.npy"  # Path to feature matrix (M, N)
LABELS = "data/labels.npy"      # Path to labels, for visualization (M,)

In [ ]:
import numpy


features = numpy.load(FEATURES)
y = numpy.load(LABELS)

## PCA

In [ ]:
N_COMPONENTS = 50     # Feature dimension after PCA
SCALE = False         # Center and scale features before PCA, optional

In [ ]:
import sklearn.decomposition
import sklearn.preprocessing


pca = sklearn.decomposition.PCA(n_components=N_COMPONENTS, svd_solver="full")

if SCALE:
    features_scaled = sklearn.preprocessing.scale(features)
else:
    features_scaled = features
    
pca.fit(features)

features_pca = pca.transform(features)

## tSNE

In [ ]:
# Parameters for tSNE. See documentation for details:
#
# http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html

OPTIONS = {
    "init": "random",
    "perplexity": 30,
    "random_state": numpy.random.randint(256),
    "verbose": 2
}

In [ ]:
from sklearn import manifold


tsne = manifold.TSNE(**OPTIONS)

%time X_embedded = tsne.fit_transform(features_pca)

## Visualization with image overlays

In [ ]:
import matplotlib.offsetbox
import matplotlib.pyplot
import numpy
import skimage.exposure
import skimage.transform

In [ ]:
X = numpy.load("data/cells.npy")
classes = numpy.load("data/classes.npy")
subsets = numpy.load("data/subsets.npy")

In [ ]:
# Scatter plot of a tSNE embedding.
#
# ax is the matplotlib axes for display.
# X_embedded is the tSNE embedding.
# y is a list of labels.
#
# alpha is the transparency of the plotted points in the range (0, 1) (default: 1.0).
# cmap is a function that takes a label as a parameter and 
#      returns a color (default: matplotlib.cm.tab10).
# labels is a list of label names, indexed by y, optional (default: None).
# marker is the point style (https://matplotlib.org/api/markers_api.html#module-matplotlib.markers) (default: "o").
# show_label is True enables a color-coded legend.
def plot_tsne(ax, X_embedded, y, alpha=1.0, cmap=matplotlib.cm.tab10, labels=None, marker="o", show_label=True):
    for label in numpy.unique(y):        
        ax.scatter(
            X_embedded[y == label][:, 0],
            X_embedded[y == label][:, 1],
            alpha=alpha,
            c=cmap(label),
            label=(label if labels is None else labels[label]) if show_label else None,
            marker=marker
        )

In [ ]:
# Select images to overlay on a tSNE plot. The images are selected by 
# maximizing their Euclidean squareddistance in the embedding.
#
# X_embedded is the tSNE embedding.
#
# init_dist is the initial distance between two points. Recommended value is the
#           maximum absolutely value of X_embedded (default: 12).
# min_dist is the minimum distance between points for an image to be included
#          in display. Smaller values result in more selected images (default: 10).
# seed is the random seed used in initializing the image selection. If None, a
#      random value is used.
#
# Returns a list of indices of points which are at least min_dist apart in
# the embedded space.
def sample(X_embedded, init_dist=12.0, min_dist=10.0, seed=None):
    shown_images = numpy.array([[init_dist, init_dist]])
    
    indices = numpy.arange(X_embedded.shape[0])
    
    if seed is not None:
        numpy.random.seed(seed)
    
    numpy.random.shuffle(indices)
    
    shown_indices = []
    
    for index in indices:
        distance = numpy.sum((X_embedded[index] - shown_images) ** 2, 1)
        
        if numpy.min(distance) < min_dist:
            continue
            
        shown_images = numpy.r_[shown_images, [X_embedded[index]]]

        shown_indices += [index]
    
    return shown_indices

In [ ]:
# Display images at the position of their tSNE embedding.
#
# ax is the matplotlib axes for display.
# X is a list of images.
# y is a list of labels.
# X_embedded is the tSNE embedding.
#
# cmap is a function that takes a label as a parameter and 
#      returns a color (default: matplotlib.cm.tab10).
# processing_fn is a function which takes an image as input and
#               returns an image as output.
# scale is a fraction by which to scale the displayed images.
#       Values less than 1.0 will downscale, greater than 1.0 will
#       upscale (default: 0.5).
def overlay(ax, X, y, X_embedded, cmap=matplotlib.cm.tab10, processing_fn=None, scale=0.5):
    for Xi, yi, pos in zip(X, y, X_embedded):
        if processing_fn:
            Xi = processing_fn(Xi)
        
        # bboxprops: https://matplotlib.org/devdocs/api/_as_gen/matplotlib.patches.FancyBboxPatch.html
        image_box = matplotlib.offsetbox.AnnotationBbox(
            matplotlib.offsetbox.OffsetImage(
                skimage.transform.rescale(Xi, scale=scale),
            ),
            pos,
            bboxprops=dict(
                edgecolor=cmap(yi),
                linewidth=8
            ),
            pad=0
        )

        ax.add_artist(image_box)
        

# Convert a multi-channel image to RGB.
#
# img is a multi-channel floating point image.
#
# r is a list of channels in img to convert to "red" (default: [0]).
# g is a list of channels in img to convert to "green" (default: [1]).
# b is a list of channels in img to convert to "blue" (default: [2]).
#
# Multiple channels are combined by summing. The final image
# is rescaled to the range (0, 1) for compatibility with display.
# 
# E.g., to_rgb(x, r=[1, 4], g=[2, 3], b=[0])
def to_rgb(img, r=[0], g=[1], b=[2]):
    rgb_img = numpy.zeros(img.shape[:-1] + (3,), dtype=numpy.float32)
    
    rgb_img[:, :, 0] = numpy.sum(img[:, :, r], axis=-1)
    rgb_img[:, :, 1] = numpy.sum(img[:, :, g], axis=-1)
    rgb_img[:, :, 2] = numpy.sum(img[:, :, b], axis=-1)
    
    return skimage.exposure.rescale_intensity(rgb_img, out_range=(0, 1))

In [ ]:
figure = matplotlib.pyplot.figure(figsize=(16, 16))

ax = matplotlib.pyplot.axes(frameon=False)

ax.set_xticks([])
ax.set_yticks([])

plot_tsne(ax, 
    X_embedded[subsets == "o"], 
    y[subsets == "o"], 
    labels=classes, 
    marker="o"
)

plot_tsne(
    ax, 
    X_embedded[subsets == "^"], 
    y[subsets == "^"], 
    alpha=0.5,
    marker="o", 
    show_label=False
)

ax.legend();  # use only if show_label is True for some plot_tsne

# use only if image overlays are wanted
samples = sample(X_embedded)

overlay(
    ax,
    X[samples], 
    y[samples], 
    X_embedded[samples],
    processing_fn=lambda x: to_rgb(x, r=[1, 4], g=[2, 3], b=[0])
)